In [1]:
# Modelando o 'churn'

In [3]:
# importando as bibliotecas

# tratamento de dados
import pandas as pd
import numpy as np

# visualização de dados
import matplotlib.pyplot as plt

# modelagem dos dados
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

In [5]:
# carregando os dados
features = pd.read_csv('data/orange_small_train.data', sep='\t', na_filter=False)
outcome = pd.read_csv('data/orange_small_train_churn.labels', header=None)


In [6]:
# tamanho dos dados
features.shape

(50000, 230)

In [7]:
# primeiras linhas dos dados
features.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230
0,,,,,,1526,7,,,,...,oslk,fXVEsaq,jySVZNlOJy,,,xb3V,RAYp,F2FyR07IdsN7I,,
1,,,,,,525,0,,,,...,oslk,2Kb5FSF,LM8l689qOp,,,fKCe,RAYp,F2FyR07IdsN7I,,
2,,,,,,5236,7,,,,...,Al6ZaUT,NKv4yOc,jySVZNlOJy,,kG3k,Qu4f,02N6s8f,ib5G6X1eUxUn6,am7c,
3,,,,,,,0,,,,...,oslk,CE7uk3u,LM8l689qOp,,,FSa2,RAYp,F2FyR07IdsN7I,,
4,,,,,,1029,7,,,,...,oslk,1J2cvxe,LM8l689qOp,,kG3k,FSa2,RAYp,F2FyR07IdsN7I,mj86,


In [8]:
outcome.shape

(50000, 1)

In [9]:
outcome.head()

,0
0,-1
1,1
2,-1
3,-1
4,-1


In [10]:
np.unique(outcome)

array([-1,  1], dtype=int64)